# Curating a genome scale model

This notebook has been tested on [jprime.lbl.gov](jprime.lbl.gov) with the biodesign_3.7 kernel.

It starts with the model that gets output by the annotation_gr.ipynb notebook.

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from IPython.display import IFrame
import numpy as np
import pandas as pd
import json
import urllib
import cobra
import cplex
import os
import requests
import collections
import itertools

import cobra.test
from cobra.flux_analysis import gapfill

# Getting and preparing the genome-scale model

## Load *R.opacus* NCBI model generated by CarveMe

In [2]:
model = cobra.io.read_sbml_model("../GSMs/Ropacus_annotated_curated.xml")
model

Name,ropacus_curated_with_5_deleted_reactions
Memory address,0x07f79a05d5850
Number of metabolites,1581
Number of reactions,2380
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, periplasm, extracellular space"


## Starting MEMOTE Output
Need to update

In [3]:
# IFrame('memotes/ropacus_curated_with_5_deleted_reactions.htm', 1500, 800)

# Look into Various methods to play with the GSM

#### follow simulating FBA documentation page 

In [4]:
model.objective = "Growth"
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 10.0
    model.medium = medium
    solution = model.optimize()
    print(solution)

<Solution 0.792 at 0x7f7a7343b1d0>


In [5]:
print(model.metabolites.get_by_id("pydam_c").name)
for r in model.metabolites.get_by_id("pydam_c").reactions:
    print(r.name, r)

Pyridoxamine
Pyridoxal oxidase PYDXO: 2.0 h2o_c + nh4_c + 0.5 o2_c + pydx_c <-- 2.0 h2o2_c + pydam_c
Pyridoxamine import PYDAMtpp: pydam_p --> pydam_c


### These are reactions in the reverse direction. Why are they there?

In [6]:
for r in [r for r in model.reactions if ('-->' not in r.reaction and '<=>' not in r.reaction)]:
    print(r)

ACACT5r: accoa_c + dcacoa_c <-- 3oddcoa_c + coa_c
BZSCD: bzsuccoa_c + fad_c <-- fadh2_c + phitcoa_c
MGt5: mg2_c <-- mg2_e
PPGKr: glc__D_c + pi_c <-- g6p_c + h2o_c
PYDXO: 2.0 h2o_c + nh4_c + 0.5 o2_c + pydx_c <-- 2.0 h2o2_c + pydam_c
SUCD1: fad_c + succ_c <-- fadh2_c + fum_c
SULR_1: 3.0 h2o_c + h2s_c + 3.0 nadp_c <-- 5.0 h_c + 3.0 nadph_c + so3_c


In [7]:
print(model.metabolites.get_by_id("fmn_c").name)
for r in model.metabolites.get_by_id("fmn_c").reactions:
    print(r.name, r)

FMN
FMN reductase  NADPH dependent  FMNRy: fmn_c + h_c + nadph_c --> fmnRD_c + nadp_c
FMNH2-dependent monooxygenase FDMO: fmnh2_c + isetac_c + o2_c --> fmn_c + gcald_c + h2o_c + h_c + so3_c
FMN reductase FMNRx: fmn_c + h_c + nadh_c --> fmnh2_c + nad_c
FMN dependent monooxygenase  taurine  FDMO4_1: fmnRD_c + o2_c + taur_c --> amacald_c + fmn_c + h2o_c + h_c + so3_c
FMN dependent monooxygenase  hexanesulfonate  FDMO3_1: fmnRD_c + hexs_c + o2_c --> fmn_c + h2o_c + h_c + hxal_c + so3_c
FmnRD dependent monooxygenase  methanesulfonate  FDMO_1: fmnRD_c + mso3_c + o2_c --> fald_c + fmn_c + h2o_c + h_c + so3_c
FMN dependent monooxygenase  butanesulfonate  FDMO2_1: buts_c + fmnRD_c + o2_c --> btal_c + fmn_c + h2o_c + h_c + so3_c
FMNH2-dependent monooxygenase (ethanesulfonate) FDMO3: ethso3_c + fmnh2_c + o2_c --> acald_c + fmn_c + h2o_c + h_c + so3_c
FMN adenylyltransferase FMNAT: atp_c + fmn_c + h_c --> fad_c + ppi_c
FMN dependent monooxygenase  sulfoacetate  FDMO5_1: fmnRD_c + o2_c + sula_c -->

What is the difference between a sink and an exchange?

Exchange reactions - are reactions that move metabolites across in silico compartments. These in silico compartments are representive of intra- and inter- cellular membranes.<br><br>
Sink reactions - The metabolites, produced in reactions that are outside of an ambit of the system or in unknown reactions, are supplied to the network with reversible sink reactions.<br><br>
Demand reactions - Irreversible reactions added to the model to consume metabolites that are deposited in the system.

In [8]:
for r in model.boundary:
    print(r)

EX_12ppd__S_e: 12ppd__S_e --> 
EX_glc__D_e: glc__D_e <=> 
EX_h2o_e: h2o_e <=> 
EX_14glucan_e: 14glucan_e --> 
EX_h_e: h_e <=> 
EX_hco3_e: hco3_e --> 
EX_2hxmp_e: 2hxmp_e --> 
EX_2pglyc_e: 2pglyc_e --> 
EX_34dhcinm_e: 34dhcinm_e --> 
EX_leu__L_e: leu__L_e --> 
EX_3hoxpac_e: 3hoxpac_e --> 
EX_3mb_e: 3mb_e --> 
EX_4hbz_e: 4hbz_e --> 
EX_ala__L_e: ala__L_e --> 
EX_cl_e: cl_e <=> 
EX_LalaDgluMdapDala_e: LalaDgluMdapDala_e --> 
EX_5mtr_e: 5mtr_e --> 
EX_arab__L_e: arab__L_e --> 
EX_4abut_e: 4abut_e --> 
EX_acac_e: acac_e --> 
EX_acald_e: acald_e --> 
EX_acgam_e: acgam_e --> 
EX_ac_e: ac_e --> 
EX_chol_e: chol_e --> 
EX_acmana_e: acmana_e --> 
EX_acnam_e: acnam_e --> 
EX_pi_e: pi_e <=> 
EX_acser_e: acser_e --> 
EX_actn__R_e: actn__R_e --> 
EX_acysbmn_e: acysbmn_e --> 
EX_adn_e: adn_e --> 
EX_ins_e: ins_e --> 
EX_nh4_e: nh4_e <=> 
EX_ad_e: ad_e --> 
EX_akg_e: akg_e --> 
EX_mal__L_e: mal__L_e --> 
EX_asn__L_e: asn__L_e --> 
EX_ala__D_e: ala__D_e --> 
EX_gln__L_e: gln__L_e --> 
EX_gly_e: gly_e -

In [11]:
for m in model.metabolites:
    if 'phenol' in m.name:
        print (m.name)

2  Hydroxymethyl phenol C7H8O2
2  Hydroxymethyl phenol C7H8O2
2-Octaprenyl-6-hydroxyphenol
2-Octaprenylphenol


In [12]:
for r in model.reactions:
    if 'phenol' in r.name:
        print (r.name)

2  hydroxymethyl phenol transport inout via proton symport
2-Octaprenylphenol hydroxylase
